In [348]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [349]:
file = 'data_sets/climate_reg_drought.csv'

In [350]:
drought_df = pd.read_csv(file, encoding="ISO-8859-1").dropna()
drought_df['Region'] = drought_df['Region'].replace(to_replace='High Plains', value=105)
drought_df['Region'] = drought_df['Region'].replace(to_replace='Midwest', value=102)
drought_df['Region'] = drought_df['Region'].replace(to_replace='Northeast', value=101)
drought_df['Region'] = drought_df['Region'].replace(to_replace='South', value=106)
drought_df['Region'] = drought_df['Region'].replace(to_replace='Southeast', value=104)
drought_df['Region'] = drought_df['Region'].replace(to_replace='West', value=109)
drought_df

,MapDate,Region,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID
0,20210831,105,19.70,80.30,61.28,42.33,21.41,1.58,8/31/2021,9/6/2021,1
1,20210831,102,61.70,38.30,24.54,12.68,6.38,1.15,8/31/2021,9/6/2021,1
2,20210831,101,77.08,22.92,12.13,1.54,0.00,0.00,8/31/2021,9/6/2021,1
3,20210831,106,92.11,7.89,1.27,0.10,0.00,0.00,8/31/2021,9/6/2021,1
4,20210831,104,94.80,5.20,0.69,0.00,0.00,0.00,8/31/2021,9/6/2021,1
...,...,...,...,...,...,...,...,...,...,...,...
3445,20100831,102,71.80,28.20,6.97,0.87,0.00,0.00,8/31/2010,9/6/2010,1
3446,20100831,101,60.07,39.93,10.37,1.50,0.00,0.00,8/31/2010,9/6/2010,1
3447,20100831,106,49.95,50.05,21.81,3.45,0.83,0.00,8/31/2010,9/6/2010,1
3448,20100831,104,56.40,43.60,13.42,4.40,0.00,0.00,8/31/2010,9/6/2010,1


In [370]:
none_df = drought_df[['Region','None','ValidStart']].copy()
none_df['ValidStart'] = pd.to_datetime(none_df['ValidStart'])
indexnames = none_df[(none_df['None'] == 0)].index
none_df.drop(indexnames , inplace=True)
none_df
per = none_df.ValidStart.dt.to_period("M")
none_region = none_df.groupby([per,'Region']).count()
none_region.head(30)
# none_region = none_df.groupby(['Region']).size().reset_index(name='counts')
# none_region


None  ValidStart
ValidStart Region                  
2010-08    101        1           1
           102        1           1
           104        1           1
           105        1           1
           106        1           1
           109        1           1
2010-09    101        4           4
           102        4           4
           104        4           4
           105        4           4
           106        4           4
           109        4           4
2010-10    101        4           4
           102        4           4
           104        4           4
           105        4           4
           106        4           4
           109        4           4
2010-11    101        5           5
           102        5           5
           104        5           5
           105        5           5
           106        5           5
           109        5           5
2010-12    101        4           4
           102        4           4
           104        4           4
           105        4           4
           106        4           4
           109        4           4

In [371]:
none_region.rename(columns={'ValidStart':'count'},inplace=True)
none_region = none_region.reset_index()
none_region.head(20)

,ValidStart,Region,None,count
0,2010-08,101,1,1
1,2010-08,102,1,1
2,2010-08,104,1,1
3,2010-08,105,1,1
4,2010-08,106,1,1
5,2010-08,109,1,1
6,2010-09,101,4,4
7,2010-09,102,4,4
8,2010-09,104,4,4
9,2010-09,105,4,4


In [372]:
none_region

,ValidStart,Region,None,count
0,2010-08,101,1,1
1,2010-08,102,1,1
2,2010-08,104,1,1
3,2010-08,105,1,1
4,2010-08,106,1,1
...,...,...,...,...
792,2021-08,102,5,5
793,2021-08,104,5,5
794,2021-08,105,5,5
795,2021-08,106,5,5


In [373]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

In [378]:
train_features = none_region.iloc[:,1:3].sample(frac=0.8, random_state=0)
test_features = none_region.iloc[:,1:3].drop(train_features.index)
#train_labels = train_features.pop('None')
test_labels = test_features.to_numpy()
#train_features.head(30)
#test_features.head(30)
#test_labels.to_numpy()
test_labels
#train_labels


array([[105,   4],
       [109,   4],
       [102,   5],
       [109,   5],
       [106,   4],
       [104,   4],
       [101,   4],
       [109,   4],
       [101,   5],
       [102,   5],
       [109,   5],
       [109,   4],
       [105,   5],
       [106,   5],
       [106,   4],
       [101,   5],
       [104,   4],
       [106,   4],
       [101,   4],
       [104,   4],
       [105,   4],
       [102,   5],
       [106,   5],
       [109,   5],
       [104,   4],
       [105,   4],
       [102,   4],
       [109,   4],
       [105,   4],
       [104,   5],
       [106,   5],
       [109,   5],
       [102,   5],
       [109,   5],
       [105,   4],
       [106,   4],
       [102,   4],
       [104,   4],
       [102,   5],
       [106,   5],
       [102,   4],
       [105,   4],
       [106,   4],
       [109,   4],
       [102,   5],
       [101,   4],
       [106,   4],
       [101,   4],
       [101,   5],
       [109,   4],
       [101,   5],
       [109,   5],
       [105,

In [379]:
normalizer = preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))

In [380]:
model = build_and_compile_model(normalizer)
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_17 (Normalizat (None, 2)                 5         
_________________________________________________________________
dense_44 (Dense)             (None, 64)                192       
_________________________________________________________________
dense_45 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_46 (Dense)             (None, 1)                 65        
Total params: 4,422
Trainable params: 4,417
Non-trainable params: 5
_________________________________________________________________


In [381]:
%%time
history = model.fit(
    train_features, train_labels,
    validation_data=(test_features, test_labels),
    verbose=0, epochs=2500)

Wall time: 49.7 s


In [359]:
test_predictions = model.predict(test_features).flatten()
test_predictions
#test_labels

array([0.962583  , 0.96422786, 0.96738523, 0.96899384, 0.9706487 ,
       1.0036985 , 3.9851828 , 3.9851732 , 3.9836528 , 3.9836292 ,
       3.9851432 , 3.9872653 , 3.9851828 , 3.9851732 , 3.9836528 ,
       3.9836292 , 3.9851432 , 3.9872653 , 4.982158  , 4.9828997 ,
       4.9843845 , 4.9851274 , 4.985871  , 4.9875903 , 3.9851828 ,
       3.9851732 , 3.9836528 , 3.9836292 , 3.9851432 , 3.9872653 ,
       3.9851825 , 3.9851735 , 3.9836528 , 3.9836292 , 3.9851432 ,
       3.9872653 , 3.9851828 , 3.9851732 , 3.9836528 , 3.9836292 ,
       3.9851432 , 3.9872653 , 4.982158  , 4.9828997 , 4.9843845 ,
       4.9851274 , 4.985871  , 4.9875903 , 3.9851828 , 3.9851732 ,
       3.9836528 , 3.9836292 , 3.9851432 , 3.9872653 , 4.982158  ,
       4.9828997 , 4.9843845 , 4.9851274 , 4.985871  , 4.9875903 ,
       3.9851828 , 3.9851732 , 3.9836528 , 3.983629  , 3.9851432 ,
       3.9872653 , 3.9851828 , 3.9851732 , 3.9836528 , 3.9836292 ,
       3.9851432 , 3.9872653 , 4.982158  , 4.9828997 , 4.98438

In [360]:
plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values percent area [D0 - No drought]')
plt.ylabel('Predictions percent area [D0 - No drought]')

lims = [0, 1250]
plt.xlim(lims)
plt.ylim(lims)
mpl.style.use('seaborn')
plt.title('Climate Region US Predicted vs Actual [No Drought]')
_ = plt.plot(lims, lims, 'C2')
#plt.savefig('figures/conus_predicted.png')
plt.show()

ValueError: x and y must be the same size